In [1]:
my_token = '6534296374:AAEUeY4HcskIHDcjRjD6rgusZQkPmmO2IEk'

### Чат бот для изучения возможностей "затравочного" программирования на основе rugpt3

In [ ]:
!pip install update gensim

In [ ]:
!pip install dialogflow gensim annoy tqdm stop_words pymorphy2 python-telegram-bot==13.3

In [ ]:
!pip install pandarallelb

In [2]:
import os
from telegram.ext  import Updater, CommandHandler, MessageHandler, Filters
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
#from gensim.models import Word2Vec, FastText


from collections.abc import Mapping
import pickle
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
import pickle
    
tqdm.pandas()

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [3]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import math
import pandas as pd

In [4]:
#загрузка моделей из hugging face
tokenizer = GPT2Tokenizer.from_pretrained("sberbank-ai/rugpt3large_based_on_gpt2")
gpt3_large = GPT2LMHeadModel.from_pretrained("sberbank-ai/rugpt3large_based_on_gpt2")
#gpt3_large = gpt3_large.to(device)

In [5]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
def generated_text(text, model, tokenizer):

    input_ids = tokenizer.encode(text, return_tensors="pt").to(device)
   
    model.eval()
    with torch.no_grad():
        out = model.generate(input_ids, 
                            do_sample=True,
                            num_beams=3,
                            temperature=0.8,
                            top_p=0.5,
                            max_length=55,
                            pad_token_id=tokenizer.eos_token_id
                            )
    generated_text = list(map(tokenizer.decode, out))[0]
    return generated_text

In [7]:
def calculate_perplexity(sentence, model, tokenizer, conatation, answer ):

    ppl_values = []

    for i in conatation:
        encodings = tokenizer(i + ' ' + sentence, return_tensors='pt')
        input_ids = encodings.input_ids.to(device)
        with torch.no_grad():
              outputs = gpt3_large(input_ids=input_ids, labels=input_ids)
        loss = outputs.loss
        ppl = math.exp(loss.item() * input_ids.size(1))
        ppl_values.append(ppl)
     
    n = min(enumerate(ppl_values),key=lambda x: x[1])[0] 
    return answer[n], n
   

In [ ]:
# Функции страха: 

# Сообщает об опасности.

# Побуждает убежать или спрятаться.


# Гнев
# Побуждает к активной защите от опасности.

# Помогает преодолеть препятствие на пути к цели.

# Придаёт сил и уверенности.

# Причины печали:

# Разлука и утрата.

# Разочарование, крушение надежд.

# Неудача в достижении цели.



# Функции отвращения:

# Регулирует дистанцию.

# Устанавливает границы, в том числе в сексуальном поведении и гигиене.

# Отвергает всё «чужеродное» в интеллектуальной, эмоциональной, ценностной и телесной, сферах.

# Предупреждает попадание невкусного, чужеродного.

# Помогает поддерживать гигиену, санитарные нормы.

# Помогает избегать сомнительных ситуаций. 



# Функции радости:

# Подтверждает доверительность в отношениях с близкими.

# Подтверждает правильность выбранного пути.

# Подтверждает удовлетворение потребностей.

# Лечит — повышает иммунитет.

# Мотивирует продолжать заниматься тем, чем мы занимаемся.


In [9]:
def gen_answer(text, gpt3_large, tokenizer):
    
    conatation = [' Функции радости:', 
                            'Функции отвращения:', 
                             'Причины печали:'  ,
                              'Защита от опасности:' ]
                  
    answer = [' Очевидно у тебя все хорошо.)))' ,  
               'Надо соблюдать дистанцию.',
              ' Разочарование, крушение надежд.(((', 
               'Присядь и прикройся.']                     
 
    triger = [' Подтверждает правильность выбранного пути.' ,  
               'Устанавливает границы, в том числе в сексуальном поведении и гигиене.',
              ' Неудача в достижении цели.', 
               ' Сообщает об опасности.']   

    
    answer_out, n = calculate_perplexity(text, gpt3_large, tokenizer, conatation, answer )
  
    #Продолжим разговор
    answer_out_2 = generated_text(triger[n], gpt3_large, tokenizer)
    
    
    return answer_out + ' ' + answer_out_2[len(triger[n]):]

In [ ]:
gen_answer("Ваш чат никуда не годится", gpt3_large, tokenizer)

In [ ]:
gen_answer("Прекрасный чат, зайду еще.", gpt3_large, tokenizer)

In [ ]:
updater = Updater(my_token, use_context=True) # Токен API к Telegram
dispatcher = updater.dispatcher

def startCommand(update, context):
    context.bot.send_message(chat_id=update.message.chat_id, text='Привет, давай пообщаемся?')
    
def textMessage(update, context):
    context.bot.send_message(chat_id=update.message.chat_id, text="...")
    
    answer = gen_answer(update.message.text, gpt3_large, tokenizer)
    answer_gen =  generated_text(update.message.text, gpt3_large, tokenizer)
    
    context.bot.send_message(chat_id=update.message.chat_id, text=answer)
    context.bot.send_message(chat_id=update.message.chat_id, text=answer_gen)
#     input_txt = preprocess_txt(update.message.text)
#     vect = vectorizer.transform([" ".join(input_txt)])
#     prediction = lr.predict(vect)
    
#     if prediction[0] == 1:
#         vect_ft = embed_txt(input_txt, idfs, midf)
#         ft_index_shop_val = ft_index_shop.get_nns_by_vector(vect_ft, 5)
#         for item in ft_index_shop_val:
#             title, image = index_map_shop[item]
#             context.bot.send_message(chat_id=update.message.chat_id, text="title: {} image: {}".format(title, image))
#         return
#     vect_ft = embed_txt(input_txt, {}, 1)
#     ft_index_val, distances = ft_index.get_nns_by_vector(vect_ft, 1, include_distances=True)
#     if distances[0] > 2.5:
#         print(distances[0])
#         context.bot.send_message(chat_id=update.message.chat_id, text="Моя твоя не понимать")
#         return
#     context.bot.send_message(chat_id=update.message.chat_id, text=index_map[ft_index_val[0]])
        
start_command_handler = CommandHandler('start', startCommand)
text_message_handler = MessageHandler(Filters.text, textMessage)
dispatcher.add_handler(start_command_handler)
dispatcher.add_handler(text_message_handler)
updater.start_polling(clean=True)
updater.idle()

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
